In [1]:
import sqlite3
import csv

In [2]:
con = sqlite3.connect('correct_noun_phrases_2.db')

In [4]:
def get_pattern_trees(con):
    cur = con.cursor()
    cur.execute("SELECT pattern_tree, count(*) FROM correct_phrase_patterns_2 GROUP BY pattern_tree ORDER BY count(*) DESC")
    
    rows = cur.fetchall()
    return rows

def get_patterns_ner(con):
    con.cursor()
    cur.execute("SELECT pattern_ner, count(*) FROM correct_phrase_patterns_2 GROUP BY pattern_ner ORDER BY count(*) DESC")
    
    rows = cur.fetchall()
    
    return rows

def get_patterns_pos(con):
    cur = con.cursor()
    cur.execute("SELECT pattern_pos, count(*) FROM correct_phrase_patterns_2 GROUP BY pattern_pos ORDER BY count(*) DESC")

    rows = cur.fetchall()
    return rows

def get_patterns_form(con):
    cur = con.cursor()
    cur.execute("SELECT pattern_form, count(*) FROM correct_phrase_patterns_2 GROUP BY pattern_form ORDER BY count(*) DESC")

    rows = cur.fetchall()
    return rows

In [19]:
def get_pattern(con, col_name):
    cur = con.cursor()
    cur.execute(f"SELECT {col_name}, count(*) FROM correct_phrase_patterns_2 GROUP BY {col_name} ORDER BY count(*) DESC")

    rows = cur.fetchall()
    return rows

In [20]:
# criteria: tree AND pos_sequence
# looking for ner_sequence x criteria
# tree is atomic tree
con = sqlite3.connect('correct_noun_phrases_2.db')
cur = con.cursor()

patterns = []
pattern_atomic_trees = get_pattern(con, "atomic_pattern")
pattern_trees = get_pattern(con, "pattern_tree")
patterns_pos = get_pattern(con, "pattern_pos")

for pattern_atomic_tree in pattern_atomic_trees:
    for pattern_tree in pattern_trees:
        for pattern_pos in patterns_pos:
            cur.execute("SELECT * FROM correct_phrase_patterns_2 WHERE atomic_pattern = ? AND pattern_tree = ? AND pattern_pos = ?", (pattern_atomic_tree[0], pattern_tree[0], pattern_pos[0]))
            criteria_freq = len(cur.fetchall())
            
            cur.execute("SELECT pattern_ner, count(*) FROM correct_phrase_patterns_2 WHERE atomic_pattern =  ? AND pattern_tree = ? AND pattern_pos = ? GROUP BY pattern_ner ORDER BY count(*) DESC", (pattern_atomic_tree[0], pattern_tree[0], pattern_pos[0]))
            pattern_ner_freqs = cur.fetchall()
                
            for pattern_ner in pattern_ner_freqs:
                if pattern_ner[1] > 1:
                    patterns.append([pattern_ner[0], pattern_atomic_tree[0], pattern_tree[0], pattern_pos[0], pattern_ner[1], pattern_ner[1]/criteria_freq])

print(patterns)

[['OTHER-OTHER-OTHER', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-S', 1206, 0.9757281553398058], ['ORG-OTHER-OTHER', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-S', 30, 0.024271844660194174], ['OTHER-OTHER-OTHER', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-H', 66, 0.55], ['OTHER-OTHER-PER', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-H', 30, 0.25], ['OTHER-OTHER-LOC', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-H', 18, 0.15], ['OTHER-OTHER-ORG', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-H', 6, 0.05], ['LOC-OTHER-OTHER', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'H-S-S', 276, 0.7076923076923077], ['ORG-OTHER-OTHER', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'H-S-S', 48, 0.12307692307692308], ['OTHER-OTHER-OTHER', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'H-S-S', 42, 0.1076923076923077], ['LOC-LOC-OTHER', '1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 nmod', 'H-S-S', 12, 0.030769230769230

In [22]:
con.close()

In [23]:
len(patterns)

264

In [24]:
with open('indicator_patterns_ner_tree_pos_updated.csv', encoding='UTF-8') as csv_file:
    reader = csv.DictReader(csv_file)
    rows = []
    for row in reader:
        rows.append(row)

In [27]:
rows[1]

{'NER_pattern': 'OTHER-OTHER',
 'tree': '1 2 nmod,2 0 root',
 'POS_pattern': 'S-S',
 'count': '6748',
 '%': '0.96',
 'is_correct': 'T',
 'mistake_type': '-',
 'error_mask': '0-0',
 'correction': '-'}

In [30]:
comparisons = []
for row in rows:
    tree_pat = row['tree'].split(',')
    pos_pat = row['POS_pattern'].split('-')
    ner_pat = row['NER_pattern'].split('-')
    
    for pat in patterns:
        if pat[1].split(',') == tree_pat:
            tree_pat2 = pat[2].split(',')
            pos_pat2 = pat[3].split('-')
            ner_pat2 = pat[0].split('-')
            
            common_tree = []
            common_pos = []
            common_ner = []
            for i in range(len(tree_pat2)):
                if tree_pat2[i] in tree_pat:
                    common_tree.append(tree_pat2[i])
                    common_pos.append(pos_pat2[i])
                    common_ner.append(ner_pat2[i])
                    
            if common_tree == tree_pat and common_pos == pos_pat and common_ner == ner_pat:
                comparisons.append([','.join(tree_pat), '-'.join(pos_pat), '-'.join(ner_pat), row['count'], row['%'], ','.join(tree_pat2), '-'.join(pos_pat2), '-'.join(ner_pat2), pat[4], pat[5]])
                    

In [31]:
len(comparisons)

256

In [32]:
for row in comparisons:
    print(row)

['1 2 nmod,2 0 root', 'S-S', 'OTHER-OTHER', '6748', '0.96', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-S', 'OTHER-OTHER-OTHER', 1206, 0.9757281553398058]
['1 2 nmod,2 0 root', 'S-S', 'OTHER-OTHER', '6748', '0.96', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-H', 'OTHER-OTHER-OTHER', 66, 0.55]
['1 2 nmod,2 0 root', 'S-S', 'OTHER-OTHER', '6748', '0.96', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-H', 'OTHER-OTHER-PER', 30, 0.25]
['1 2 nmod,2 0 root', 'S-S', 'OTHER-OTHER', '6748', '0.96', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-H', 'OTHER-OTHER-LOC', 18, 0.15]
['1 2 nmod,2 0 root', 'S-S', 'OTHER-OTHER', '6748', '0.96', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-H', 'OTHER-OTHER-ORG', 6, 0.05]
['1 2 nmod,2 0 root', 'S-S', 'OTHER-OTHER', '6748', '0.96', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-Y', 'OTHER-OTHER-OTHER', 66, 0.7333333333333333]
['1 2 nmod,2 0 root', 'S-S', 'OTHER-OTHER', '6748', '0.96', '1 2 nmod,2 0 root,3 2 nmod', 'S-S-Y', 'OTHER-OTHER-ORG', 24, 0.26666666666666666]
['1 2 nmod,2 0 root', 'S-S', 'OTHER-OTHER', '6748', '0

In [33]:
with open('indicator_patterns_ner_tree_pos_added_context.csv', 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['NER_pattern', 'tree', 'POS_pattern', 'count', '%', 'NER_pattern_added_context', 'tree_added_context', 'POS_pattern_added_context', 'count_added_context', '%_added_context'])
    writer.writeheader()
    header2 = {'NER_pattern': 'string', 'tree': 'string', 'POS_pattern': 'string', 'count': 'int64', '%': 'float64', 'NER_pattern_added_context': 'string', 'tree_added_context': 'string', 'POS_pattern_added_context': 'string', 'count_added_context': 'int64', '%_added_context': 'float64'}
    writer.writerow(header2)
    for row in comparisons:
        writer.writerow({'NER_pattern': row[2], 'tree': row[0], 'POS_pattern': row[1], 'count': row[3], '%': row[4], 'NER_pattern_added_context': row[7], 'tree_added_context': row[5], 'POS_pattern_added_context': row[6], 'count_added_context': row[8], '%_added_context': row[9]})

In [34]:
with open('indicator_patterns_ner_tree_pos_added_context.csv', encoding='UTF-8') as csv_file:
    reader = csv.DictReader(csv_file)
    rows = []
    for row in reader:
        rows.append(row)

In [35]:
rows[1]

{'NER_pattern': 'OTHER-OTHER',
 'tree': '1 2 nmod,2 0 root',
 'POS_pattern': 'S-S',
 'count': '6748',
 '%': '0.96',
 'NER_pattern_added_context': 'OTHER-OTHER-OTHER',
 'tree_added_context': '1 2 nmod,2 0 root,3 2 nmod',
 'POS_pattern_added_context': 'S-S-S',
 'count_added_context': '1206',
 '%_added_context': '0.9757281553398058'}

In [44]:
improved_percent = []
for idx, row in enumerate(rows):
    if idx == 0:
        continue
    if float(row['%_added_context']) > float(row['%']) and int(row['count_added_context']) > 1:
        improved_percent.append(row)

In [45]:
len(improved_percent)

144

In [46]:
with open('indicator_patterns_ner_tree_pos_added_context_impr_%.csv', 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['NER_pattern', 'tree', 'POS_pattern', 'count', '%', 'NER_pattern_added_context', 'tree_added_context', 'POS_pattern_added_context', 'count_added_context', '%_added_context'])
    writer.writeheader()
    header2 = {'NER_pattern': 'string', 'tree': 'string', 'POS_pattern': 'string', 'count': 'int64', '%': 'float64', 'NER_pattern_added_context': 'string', 'tree_added_context': 'string', 'POS_pattern_added_context': 'string', 'count_added_context': 'int64', '%_added_context': 'float64'}
    writer.writerow(header2)
    for row in improved_percent:
        writer.writerow({'NER_pattern': row['NER_pattern'], 'tree': row['tree'], 'POS_pattern': row['POS_pattern'], 'count': row['count'], '%': row['%'], 'NER_pattern_added_context': row['NER_pattern_added_context'], 'tree_added_context': row['tree_added_context'], 'POS_pattern_added_context': row['POS_pattern_added_context'], 'count_added_context': row['count_added_context'], '%_added_context': row['%_added_context']})